# 自定义求导与Keras的结合使用
* 自定义求导主要是在模型fit的阶段进行使用
* 在fit过程中tf帮我们做了哪些事情：
    * 1.batch 遍历训练集，计算metric
        - 1.1 自动求导
    * 2.epoch 结束，遍历验证集，计算metric
        

In [1]:
import matplotlib as mlp
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras 
import warnings

warnings.filterwarnings('ignore')
print(tf.__version__)
for model in sklearn, pd, keras, np ,mlp:
    print(model.__name__, model.__version__)

2.1.0
sklearn 0.20.2
pandas 0.24.2
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
numpy 1.17.4
matplotlib 2.1.2


In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
housing = fetch_california_housing()
x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state=1)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state=2)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [3]:
# version 0.2 数据归一化
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
x_train = std.fit_transform(x_train)
x_valid = std.transform(x_valid)
x_test = std.transform(x_test)

In [9]:
# tf.keras.models.Sequential()
# 设置一部分超参数
epochs = 10
batch_size = 32
steps_per_epoch = len(x_train) // batch_size
optimizer = keras.optimizers.SGD()
metric = keras.metrics.MeanSquaredError()
def random_batch(x, y, batch_size=32):
    idx = np.random.randint(0, len(x), size=batch_size)
    return x[idx], y[idx]

# 固化模型
model = tf.keras.models.Sequential([
    keras.layers.Dense(30,activation='relu',input_shape=x_train.shape[1:]),
    keras.layers.Dense(1),
])

# 模型训练
for epoch in range(epochs):
    metric.reset_states()
    for step in range(steps_per_epoch):
        x_batch, y_batch = random_batch(x_train, y_train, batch_size)
        with tf.GradientTape() as tape:
            y_pred = model(x_batch)
            loss = tf.reduce_mean(keras.losses.mean_squared_error(y_batch, y_pred))
            metric(y_batch, y_pred)
        grads = tape.gradient(loss, model.variables)
        grads_and_vars = zip(grads, model.variables)
        optimizer.apply_gradients(grads_and_vars)
        print("\rEpoch ", epoch, "train mse:", metric.result().numpy(), end="")
    y_valid_pred = model(x_valid)
    valid_loss = tf.reduce_mean(keras.losses.mean_squared_error(y_valid, y_valid_pred))
    print("\t", "valid mse: ", valid_loss.numpy())


W0219 10:44:57.914844 4641748416 base_layer.py:1790] Layer dense_4 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Epoch  0 train mse: 1.7646081	 valid mse:  1.3964492
Epoch  1 train mse: 1.3589876	 valid mse:  1.3664284
Epoch  2 train mse: 1.3391879	 valid mse:  1.3507336
Epoch  3 train mse: 1.3215915	 valid mse:  1.3452963
Epoch  4 train mse: 1.3271685	 valid mse:  1.3430723
Epoch  5 train mse: 1.2990948	 valid mse:  1.3418753
Epoch  6 train mse: 1.3281983	 valid mse:  1.3461287
Epoch  7 train mse: 1.2779781	 valid mse:  1.341186
Epoch  8 train mse: 1.2900978	 valid mse:  1.3419476
Epoch  9 train mse: 1.2991757	 valid mse:  1.3396543
